In [5]:
import sys
import os

sys.path.append(os.path.abspath("../"))
from Backtest.log_parser import Parser
import warnings
import plotly.express as px
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore")

In [6]:
parser = Parser(
    "C:/Users/edmun/OneDrive/Desktop/2025-IMC-Global-Trading-Challenge/Tutorial/Tutorial_log.log"
)
parser_2 = Parser("C:/Users/edmun/Downloads/4e66ba88-1906-4d4e-a000-cd23df6ff957.log")

In [7]:
df = parser.get_pd()
df2 = parser_2.get_pd()
df = df[df.listing == "KELP"]
df2 = df2[df2.listing == "KELP"]

In [8]:
# Updated extract functions that return lists
def extract_bids(data):
    bids = [entry["price"] for entry in data if entry["quantity"] > 0]
    return bids


def extract_asks(data):
    asks_order = [entry["price"] for entry in data if entry["quantity"] < 0]
    return asks_order


def extract_bids_qty(data):
    bids = [entry["quantity"] for entry in data if entry["quantity"] > 0]
    return bids


def extract_asks_qty(data):
    asks_order = [entry["quantity"] for entry in data if entry["quantity"] < 0]
    return asks_order


def extract_market_bids(data):
    bids = [price for price, qty in data[0].items()]
    return bids


def extract_market_asks(data):
    asks_order = [price for price, qty in data[1].items()]
    return asks_order


def extract_market_trade_price(data):
    price = [
        entry["price"]
        for entry in data
        if entry["timestamp_now"] - 100 == entry["timestamp"]
        # or entry["timestamp"] == entry["timestamp_now"]
    ]
    return price


def extract_market_trade_qty(data):
    quantity = [
        entry["quantity"]
        for entry in data
        if entry["timestamp_now"] - 100 == entry["timestamp"]
        # or entry["timestamp"] == entry["timestamp_now"]
    ]
    return quantity


def extract_market_own_trade_buy(data):
    price = [
        entry["price"]
        for entry in data
        if entry["buyer"] == "SUBMISSION"
        and (
            entry["timestamp_now"] - 100 == entry["timestamp"]
            or entry["timestamp"] == entry["timestamp_now"]
        )
    ]
    return price


def extract_market_own_trade_sell(data):
    price = [
        entry["price"]
        for entry in data
        if entry["seller"] == "SUBMISSION"
        and (
            entry["timestamp_now"] - 100 == entry["timestamp"]
            or entry["timestamp"] == entry["timestamp_now"]
        )
    ]
    return price


def extract_market_own_trade_buy_qty(data):
    quantity = [
        entry["quantity"]
        for entry in data
        if entry["buyer"] == "SUBMISSION"
        and (
            entry["timestamp_now"] - 100 == entry["timestamp"]
            or entry["timestamp"] == entry["timestamp_now"]
        )
    ]
    return quantity


def extract_market_own_trade_sell_qty(data):
    quantity = [
        entry["quantity"]
        for entry in data
        if entry["seller"] == "SUBMISSION"
        and (
            entry["timestamp_now"] - 100 == entry["timestamp"]
            or entry["timestamp"] == entry["timestamp_now"]
        )
    ]
    return quantity


def create_dynamic_columns(row, col, max_entries, func, col_name):
    # Get the correct column data and timestamp for the current row
    values = func(row)  # Pass the timestamp from the row for filtering
    columns = {}
    for i in range(max_entries):
        if i < len(values):
            columns[f"{col_name}_{i + 1}"] = values[i]
        else:
            columns[f"{col_name}_{i + 1}"] = np.nan
    return pd.Series(columns)


# Assuming `df` is your DataFrame
def apply_dynamic_columns(df, func, col, column_name):
    # Calculate the maximum number of entries for the given function
    max_entries = 10
    # Apply the function row-wise and create dynamic columns
    df = pd.merge(
        df,
        df[col].apply(
            lambda row: create_dynamic_columns(row, col, max_entries, func, column_name)
        ),
        how="left",
        left_index=True,
        right_index=True,
    )

    return df


def convert(df):
    # Apply dynamic columns for each function
    df = apply_dynamic_columns(df, extract_bids, "orders", "my_bid_price")
    df = apply_dynamic_columns(df, extract_asks, "orders", "my_ask_price")
    df = apply_dynamic_columns(df, extract_bids_qty, "orders", "my_bid_qty")
    df = apply_dynamic_columns(df, extract_asks_qty, "orders", "my_ask_qty")
    df = apply_dynamic_columns(
        df, extract_market_bids, "orderDepths", "market_bid_order"
    )
    df = apply_dynamic_columns(
        df, extract_market_asks, "orderDepths", "market_ask_order"
    )
    df = apply_dynamic_columns(
        df, extract_market_trade_price, "marketTrades", "marketTrades_price"
    )
    df = apply_dynamic_columns(
        df, extract_market_trade_qty, "marketTrades", "marketTrades_qty"
    )
    df = apply_dynamic_columns(
        df, extract_market_own_trade_buy, "ownTrades", "ownTrades_buy_price"
    )
    df = apply_dynamic_columns(
        df, extract_market_own_trade_sell, "ownTrades", "ownTrades_sell_price"
    )
    df = apply_dynamic_columns(
        df, extract_market_own_trade_buy_qty, "ownTrades", "ownTrades_buy_qty"
    )
    df = apply_dynamic_columns(
        df, extract_market_own_trade_sell_qty, "ownTrades", "ownTrades_sell_qty"
    )
    return df


In [9]:
df = convert(df)
df2 = convert(df2)

In [10]:
df.columns

Index(['timestamp', 'listing', 'orderDepths', 'midPrice', 'orders',
       'ownTrades', 'marketTrades', 'position', 'observations', 'conversions',
       ...
       'ownTrades_sell_qty_1', 'ownTrades_sell_qty_2', 'ownTrades_sell_qty_3',
       'ownTrades_sell_qty_4', 'ownTrades_sell_qty_5', 'ownTrades_sell_qty_6',
       'ownTrades_sell_qty_7', 'ownTrades_sell_qty_8', 'ownTrades_sell_qty_9',
       'ownTrades_sell_qty_10'],
      dtype='object', length=133)

In [11]:
plot = df2[(df2["timestamp"] > 170_000) & (df2["timestamp"] < 172_000)][
    [
        "timestamp",
        "midPrice",
        'position',
        "my_bid_price_1",
        "my_bid_price_2",
        "my_bid_price_3",
        "my_ask_price_1",
        "my_ask_price_2",
        "my_ask_price_3",
        "my_bid_qty_1",
        "my_bid_qty_2",
        "my_bid_qty_3",
        "my_ask_qty_1",
        "my_ask_qty_2",
        "my_ask_qty_3",
        "market_bid_order_1",
        "market_bid_order_2",
        "market_bid_order_3",
        "market_ask_order_1",
        "market_ask_order_2",
        "market_ask_order_3",
        "marketTrades_price_1",
        "marketTrades_price_2",
        "marketTrades_price_3",
        "marketTrades_qty_1",
        "marketTrades_qty_2",
        "marketTrades_qty_3",
        "ownTrades_buy_price_1",
        "ownTrades_buy_price_2",
        "ownTrades_buy_price_3",
        "ownTrades_sell_price_1",
        "ownTrades_sell_price_2",
        "ownTrades_sell_price_3",
        "ownTrades_buy_qty_1",
        "ownTrades_buy_qty_2",
        "ownTrades_buy_qty_3",
        "ownTrades_sell_qty_1",
        "ownTrades_sell_qty_2",
        "ownTrades_sell_qty_3",
    ]
]

plot[plot.columns[1:]] = plot[plot.columns[1:]].astype(float)

px.line(plot, x="timestamp", y=list(plot.columns[1:]), height=1_000)

In [12]:
df["pos_pct"] = df["position"] / 50
df2["pos_pct"] = df2["position"] / 50
# Create a combined plot with both dataframes
fig = px.line()

# Add lines for both df and df2 to the plot
fig.add_scatter(
    x=df["timestamp"],
    y=df["pos_pct"],
    mode="lines",
    name="DF1",
    line=dict(color="blue"),
)
fig.add_scatter(
    x=df2["timestamp"],
    y=df2["pos_pct"],
    mode="lines",
    name="DF2",
    line=dict(color="red"),
)

# Show the plot
fig.show()

In [13]:
def plot(col):
    fig = px.line()

    # Add lines for both df and df2 to the plot
    fig.add_scatter(
        x=df["timestamp"],
        y=df[col],
        mode="lines",
        name="DF1",
        line=dict(color="blue"),
    )
    fig.add_scatter(
        x=df2["timestamp"],
        y=df2[col],
        mode="lines",
        name="DF2",
        line=dict(color="red"),
    )

    # Show the plot
    fig.show()

plot('profit_and_loss')